In [1]:
#tabular data imports :
import pandas as pd
import numpy as np
import env
from env import username, password, host
from pydataset import data

# visualization imports:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
# success metrics from earlier in the week: mean squared error and r^2 explained variance
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
#stats
from scipy.stats import pearsonr, spearmanr
from scipy.stats import shapiro

import wrangle as w
import warnings
warnings.filterwarnings("ignore")
import os
directory = os.getcwd()

In [2]:
df = pd.read_csv('concrete.csv')
df.head(3)

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,141.3,212.0,0.0,203.5,0.0,971.8,748.5,28,29.89
1,168.9,42.2,124.3,158.3,10.8,1080.8,796.2,14,23.51
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,28,29.22


# CURRENT DATA AND UNITS
## - Name -- Data Type -- Measurement -- Description

#### - Cement (component 1) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Blast Furnace Slag (component 2) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Fly Ash (component 3) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Water (component 4) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Superplasticizer (component 5) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Coarse Aggregate (component 6) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Fine Aggregate (component 7) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Age -- quantitative -- Day (1~365) -- Input Variable
#### - Concrete compressive strength -- quantitative -- MPa -- Output Variable

In [3]:
# Conversion factors
KG_TO_LBS = 2.20462
M3_TO_CUBIC_YARD = 1.30795
MPA_TO_PSI = 145.038

# Convert units to imperial
df['cement'] = df['cement'] * KG_TO_LBS
df['slag'] = df['slag'] * KG_TO_LBS
df['ash'] = df['ash'] * KG_TO_LBS
df['water'] = df['water'] * KG_TO_LBS
df['superplastic'] = df['superplastic'] * KG_TO_LBS 
df['coarseagg'] = df['coarseagg'] * KG_TO_LBS
df['fineagg'] = df['fineagg'] * KG_TO_LBS
df['strength'] = df['strength'] * MPA_TO_PSI

 # Add an index column named 'sample'
df.reset_index(inplace=True)
df.rename(columns={'index': 'sample'}, inplace=True)

# Set the 'sample' column as the index
df.set_index('sample', inplace=True)

df.head(3)

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
sample,,,,,,,,,
0,311.512806,467.379440,0.000000,448.640170,0.000000,2142.449716,1650.158070,28,4335.18582
1,372.360318,93.034964,274.034266,348.991346,23.809896,2382.753296,1755.318444,14,3409.84338
2,551.155000,0.000000,210.982134,413.145788,12.125410,2109.600878,1898.618744,28,4238.01036


In [4]:
concrete = pd.read_csv('concrete.csv')

In [5]:
# first iteration of function to call 
df = w.prepare_data1(concrete)

# observing the changes
df.head(3)

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,311.512806,467.379440,0.000000,448.640170,0.000000,2142.449716,1650.158070,28,4335.18582
1,372.360318,93.034964,274.034266,348.991346,23.809896,2382.753296,1755.318444,14,3409.84338
2,551.155000,0.000000,210.982134,413.145788,12.125410,2109.600878,1898.618744,28,4238.01036


In [6]:
# information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cement        1030 non-null   float64
 1   slag          1030 non-null   float64
 2   ash           1030 non-null   float64
 3   water         1030 non-null   float64
 4   superplastic  1030 non-null   float64
 5   coarseagg     1030 non-null   float64
 6   fineagg       1030 non-null   float64
 7   age           1030 non-null   int64  
 8   strength      1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


## Will be converting the units to US Imperial units

In [7]:
# Conversion factors
KG_TO_LBS = 2.20462
M3_TO_CUBIC_YARD = 1.30795
MPA_TO_PSI = 145.038

# Convert units to imperial
df['cement'] = df['cement'] * KG_TO_LBS
df['slag'] = df['slag'] * KG_TO_LBS
df['ash'] = df['ash'] * KG_TO_LBS
df['water'] = df['water'] * KG_TO_LBS
df['superplastic'] = df['superplastic'] * KG_TO_LBS
df['coarseagg'] = df['coarseagg'] * KG_TO_LBS
df['fineagg'] = df['fineagg'] * KG_TO_LBS
df['strength'] = df['strength'] * MPA_TO_PSI

# NEW DATA AND UNITS
## - Name -- Data Type -- Measurement -- Description

#### - Cement (component 1) -- quantitative -- lbs per yd³ mixture -- Input Variable
#### - Blast Furnace Slag (component 2) -- quantitative -- lbs per yd³ mixture -- Input Variable
#### - Fly Ash (component 3) -- quantitative -- lbs per yd³ mixture -- Input Variable
#### - Water (component 4) -- quantitative -- lbs per yd³ mixture -- Input Variable
#### - Superplasticizer (component 5) -- quantitative -- lbs per yd³ mixture -- Input Variable
#### - Coarse Aggregate (component 6) -- quantitative -- lbs per yd³ mixture -- Input Variable
#### - Fine Aggregate (component 7) -- quantitative -- lbs per yd³ mixture -- Input Variable
#### - Age -- quantitative -- Day (1~365) -- Input Variable
#### - Concrete compressive strength -- quantitative -- psi -- Output Variable

In [8]:
# second iteration of modifications to the original dataset, including unit conversion and the addition to a sample number index
df = w.prepare_data2(concrete)
df.head(3)

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
sample,,,,,,,,,
0,311.512806,467.379440,0.000000,448.640170,0.000000,2142.449716,1650.158070,28,4335.18582
1,372.360318,93.034964,274.034266,348.991346,23.809896,2382.753296,1755.318444,14,3409.84338
2,551.155000,0.000000,210.982134,413.145788,12.125410,2109.600878,1898.618744,28,4238.01036


In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
cement,1030.0,619.868297,230.396821,224.871240,424.113772,601.640798,771.617000,1190.494800
slag,1030.0,162.912214,190.213162,0.000000,0.000000,48.501640,315.150429,792.340428
ash,1030.0,119.464719,141.089075,0.000000,0.000000,0.000000,260.806546,441.144462
water,1030.0,400.286860,47.077937,268.522716,363.541838,407.854700,423.287040,544.541140
superplastic,1030.0,13.678918,13.170050,0.000000,0.000000,14.109568,22.487124,70.988764
coarseagg,1030.0,2144.916536,171.417922,1765.900620,2054.705840,2134.072160,2269.435828,2524.289900
fineagg,1030.0,1705.451010,176.757569,1309.544280,1611.466989,1718.501290,1816.606880,2188.305812
age,1030.0,45.662136,63.169912,1.000000,7.000000,28.000000,56.000000,365.000000
strength,1030.0,5194.965451,2422.967403,337.938540,3438.850980,4995.833910,6691.328130,11980.138800


In [10]:
# checking for null values
df.isnull().sum()

cement          0
slag            0
ash             0
water           0
superplastic    0
coarseagg       0
fineagg         0
age             0
strength        0
dtype: int64

In [11]:
# checking for possible categorical features
df.nunique()

cement          278
slag            185
ash             156
water           195
superplastic    111
coarseagg       284
fineagg         302
age              14
strength        845
dtype: int64

In [12]:
# distribution of age samples
df.age.value_counts()

28     425
3      134
7      126
56      91
14      62
90      54
100     52
180     26
91      22
365     14
270     13
360      6
120      3
1        2
Name: age, dtype: int64

In [13]:
# List of components
components = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg']

# Sum the values across the specified columns and create the new column
df['total_lbs_per_yd^3'] = df[list(components)].sum(axis=1)
df.head(3)

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength,total_lbs_per_yd^3
sample,,,,,,,,,,
0,311.512806,467.379440,0.000000,448.640170,0.000000,2142.449716,1650.158070,28,4335.18582,5020.140202
1,372.360318,93.034964,274.034266,348.991346,23.809896,2382.753296,1755.318444,14,3409.84338,5250.302530
2,551.155000,0.000000,210.982134,413.145788,12.125410,2109.600878,1898.618744,28,4238.01036,5195.627954


In [14]:
# edited the function so im reruning it once more
df = w.prepare_data3(concrete)
df.head(3)

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength,total_lbs_per_yd^3
sample,,,,,,,,,,
0,686.767362,1030.394061,0.000000,989.081092,0.000000,4723.287493,3637.971484,28,628766.680961,11068.0
1,820.913004,205.106742,604.141424,769.393301,52.491773,5253.065571,3869.810148,14,494556.864148,11575.0
2,1215.087336,0.000000,465.135432,910.829467,26.731921,4650.868288,4185.732855,28,614672.546594,11454.0


### will be adding a column that takes columns and represents them as percent in relation to total weight.

In [15]:
# List of components
components = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg']

# Calculate the percentage of each component relative to the total_lbs_per_yd^3
for component in components:
    df[component + '_pct'] = round((df[component] / df['total_lbs_per_yd^3']) * 100,2)

# Display the first few rows to verify the new columns
df.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength,total_lbs_per_yd^3,cement_pct,slag_pct,ash_pct,water_pct,superplastic_pct,coarseagg_pct,fineagg_pct
sample,,,,,,,,,,,,,,,,,
0,686.767362,1030.394061,0.000000,989.081092,0.000000,4723.287493,3637.971484,28,628766.680961,11068.0,6.20,9.31,0.00,8.94,0.00,42.68,32.87
1,820.913004,205.106742,604.141424,769.393301,52.491773,5253.065571,3869.810148,14,494556.864148,11575.0,7.09,1.77,5.22,6.65,0.45,45.38,33.43
2,1215.087336,0.000000,465.135432,910.829467,26.731921,4650.868288,4185.732855,28,614672.546594,11454.0,10.61,0.00,4.06,7.95,0.23,40.60,36.54
3,1292.852926,554.079825,0.000000,1108.159651,0.000000,4529.845589,3256.434061,28,964501.583207,10741.0,12.04,5.16,0.00,10.32,0.00,42.17,30.32
4,752.382079,891.388070,0.000000,939.505528,44.229179,5090.729903,3386.205388,28,384748.832211,11104.0,6.78,8.03,0.00,8.46,0.40,45.85,30.50
